In [ ]:
# %%
import numpy as np
import aerokit.common.defaultgas as defg
import aerokit.aero.Isentropic as Is
import aerokit.aero.ShockWave  as sw
import aerokit.aero.MassFlow   as mf
import aerokit.instance.nozzle as nz
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16
#
def new_plot(xlab, ylab):
    plt.figure(figsize=(15,8))
    plt.xlabel(xlab); plt.ylabel(ylab)
    plt.grid(which='major', linestyle='-', alpha=0.8)
    plt.grid(which='minor', linestyle=':', alpha=0.5)

target_AoAc = 4.
length      = 8.
#
# set an A/Ac law from a Mach evolution, ensure exit Mach number is consistent with As/Ac
Noz_x    = np.linspace(0., length, 400, endpoint=True)
ma_max   = mf.Mach_Sigma(target_AoAc, Mach=2.)
ma       = 1. + (ma_max-1.)*np.sin(.5*(Noz_x-1.)*np.pi/(length-1.))
Noz_AoAc = mf.Sigma_Mach(ma)
#
noz = nz.nozzle(Noz_x, Noz_AoAc)

NPR0, NPRsw, NPR1, Msub, Msh, Msup = nz._NPR_Ms_list(target_AoAc)
print("NPR limits are ", NPR0, NPRsw, NPR1, "\nwith respective Mach ", Msub, Msh, Msup)
mf.Sigma_Mach(Is.Mach_PiPs(np.array(nz._NPR_Ms_list(2.)[:3:2])))

def make_fig(Mmax=5):
    global fig, ax1, ax2
    fig, ax1 = plt.subplots(figsize=(15,8))
    ax1.set_xlabel('$x$'); ax1.set_ylabel('$P_s$, $P_t$')
    ax1.grid(which='major', linestyle='-', alpha=0.8)
    ax1.grid(which='minor', linestyle=':', alpha=0.5)
    ax1.set_ylim(0., 1.05)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Mach')
    ax2.set_ylim(0., Mmax*1.05)
    
def plot_crv(AsAc, NPR, Mach=False):
    global ax1, ax2
    noz = nz.nozzle(Noz_x, 1.+(AsAc-1)/(target_AoAc-1)*(Noz_AoAc-1.))
    noz.set_NPR(NPR)
    NPR0, NPRsw, NPR1, Msub, Msh, Msup = nz._NPR_Ms_list(AsAc)
    ax1.plot(Noz_x, noz.Ptot(),'b-')
    ax1.plot(Noz_x, noz.Ps(),'r-')
#    plt.plot(Noz_x, AsAc*Noz_x, Noz_x, NPR*Noz_x)
    ax1.plot(Noz_x[-1], 1./NPR, 'ro')
    ax1.plot([Noz_x[-1], Noz_x[-1]], [1./NPR0, 1./NPRsw], 'go')
    if Mach: 
        ax2.plot(Noz_x, noz.Mach(), 'b--')

def iplot(AsAc, NPR, Mach=False):
    global fig, ax1, ax2
    make_fig()
    plot_crv(AsAc, NPR, Mach)
    plt.show()

#iplot(target_AoAc, 1.1, Mach=False)

# %%
import ipywidgets as pyw

plt.rcParams['figure.dpi']  = 80
fig = plt.figure(figsize=(15,8))

wi_AsAc = pyw.FloatLogSlider(value=10, base=10, min=0, max=1, step=.02, description="As/Ac")
wi_NPR  = pyw.FloatLogSlider(value=2, base=10, min=0, max=2, step=.005, description='NPR')
#pyw.HBox(
#    pyw.VBox(),
#    pyw.VBox(),
#)
interactive_plot = pyw.interactive(iplot, AsAc=wi_AsAc, NPR=wi_NPR, Mach=True)
# output = interactive_plot.children[-1]
# output.layout.height = '500px'
interactive_plot


# %% [markdown]
# ## variation As/Ac
